In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from math import ceil, floor
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest, pairwise, tee
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum, auto
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set, Dict
import functools
from operator import itemgetter
np.set_printoptions(linewidth=160)

In [3]:
DAY = 10

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'7.FF7777.|77.-L-77.|F-7-.L77|7F|7-|-JF77FFF777.7FL--F-F..F.-7-F7-J.FFF7JFF7-J7..'
Split Data:
['7.FF7777.|77.-L-77.|F-7-.L77|7F|7-|-JF77FFF777.7FL--F-F..F.-7-F7-J.FFF7JFF7-J7....FFJFL.|-L7F--|-L7L-|-|-|.F77F7F-|7---L-.7-.F-7F|-L77...-.-',
 'J7L-||LL7L|7-J.|.|7LL-|.F7|FJ7----J-7-|LLJ|JJ||LJL|L|-|.|J.LL-|F7|JFFJL7LFF7|J7-L|LJ-||FL-FLJ7L-7JFLFJ.-FL.||F|7.LJ|7J.L.77JFF.L-7J.LL--7--.',
 '.77L|7-|L-L7J.L7FJ.||F|F|-F-JJ.|..|LL-L.JJ|L-7JL|FL-|-J7J-FF|L||FFJF|77|LLL7.F7JLF7.L-|JJF7..F.F|F|-J.L-LL|-|7L-|-F|J7-L-F|-777|L|F7-L-77FJ7',
 '-777||LJ.|J|.--F7.7-LL|---JJ7.F-J-|JLF|-L--|7|.-JL7F7LL----FF7.F|JL|LJ-J7.FL7-.7F||7.|.L7F77F7F77F77LF7F7LL.-JF|.F||777.|F..||FF-JFJ7..L-|JJ',
 'F|F77F--7JL|7JFF--L..FJJ7FJFFF-77||F7FL-|LF--|LJ.|-J-7J|LJFLJL7|L7---JJ.--FJL.J7FJ|.L--F---7|LJL-JL7-J|JL.|F.F|FLJ7|F7FL7L|7FLJ-J7..FF.|J.L7',
 'FLJJFL|L7-FL--FLJ7L7-L|J|-7F-|FL-|JJFJJ7F7-J.F--F-JL-|-L7F-7FJ|LL--JL7F|J7|F|.LFJFJ...|L7F7|L-7F---JJF|7.FLJ7J||.L-JJLL7JL7--JLLLF7.J|L-7F.7',
 'F|L-7F7-|-JLLJ|F|L.L7LF

In [8]:
test_data = """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ""".split("\n")

In [30]:
ASCII_DICT = {
    "F" : "╭",
    "7" : "╮",
    "L" : "╰",
    "J" : "╯",
    "|" : "│",
    "-" : "─",
    "S" : "╳",
    "." : " ",
}

In [33]:
ASCII_DICT = {
    "F" : "┏",
    "7" : "┓",
    "L" : "┗",
    "J" : "┛",
    "|" : "┃",
    "-" : "━",
    "S" : "╳",
    "." : " ",
}

In [34]:
EXIT_DICT = {
    "F" : (1,1j),
    "7" : (-1,1j),
    "L" : (1, -1j),
    "J" : (-1, -1j),
    "|" : (-1j, 1j),
    "-" : (-1,1),
    "S" : tuple(),
    "." : tuple(),
}

In [38]:
CARDINALS = (1,-1,-1j,1j)

In [72]:
class Pipes:
    def __init__(self, lines_in: List[str]) -> None:
        self.lines = lines_in
        self.nice_lines = ["".join(ASCII_DICT[x] for x in line) for line in self.lines]
        self.start, self.grid_dict, self.exit_grid = self.parse()

    def parse(self):
        grid_dict = dict()
        e_dict = dict()
        start = None
        for y, line in enumerate(self.lines):
            for x, char in enumerate(line):
                if char == "S":
                    start = x + y * 1j
                grid_dict[x + y * 1j] = char
                e_dict[x + y * 1j] = EXIT_DICT[char]

        return start, grid_dict, e_dict

    def traverse(self):
        # find first valid exit
        current = self.start
        path = [current]
        for exit_direction in CARDINALS:
            entry_direction = -exit_direction
            try:
                possible = current + exit_direction
                if entry_direction in self.exit_grid[possible]:
                    current = possible
                    exit_direction = set(self.exit_grid[possible]).difference({entry_direction}).pop()
                    break
            except KeyError:
                continue

        path.append(current)
        while True:
            entry_direction = -exit_direction
            current = current + exit_direction
            path.append(current)
            if current == self.start:
                break
            exit_direction = set(self.exit_grid[current]).difference({entry_direction}).pop()

        l = len(path)
        return [min(a, b) for a, b in zip(range(l), reversed(range(l)))]

    def __str__(self):
        return "\n".join(self.nice_lines)

    def __repr__(self) -> str:
        return str(self)

In [73]:

Pipes(test_data)


┓━┏┓━
 ┏┛┃┓
╳┛┗┗┓
┃┏━━┛
┗┛ ┗┛

In [74]:
Pipes(test_data).traverse()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [75]:
Pipes(data)

┓ ┏┏┓┓┓┓ ┃┓┓ ━┗━┓┓ ┃┏━┓━ ┗┓┓┃┓┏┃┓━┃━┛┏┓┓┏┏┏┓┓┓ ┓┏┗━━┏━┏  ┏ ━┓━┏┓━┛ ┏┏┏┓┛┏┏┓━┛┓    ┏┏┛┏┗ ┃━┗┓┏━━┃━┗┓┗━┃━┃━┃ ┏┓┓┏┓┏━┃┓━━━┗━ ┓━ ┏━┓┏┃━┗┓┓   ━ ━
┛┓┗━┃┃┗┗┓┗┃┓━┛ ┃ ┃┓┗┗━┃ ┏┓┃┏┛┓━━━━┛━┓━┃┗┗┛┃┛┛┃┃┗┛┗┃┗┃━┃ ┃┛ ┗┗━┃┏┓┃┛┏┏┛┗┓┗┏┏┓┃┛┓━┗┃┗┛━┃┃┏┗━┏┗┛┓┗━┓┛┏┗┏┛ ━┏┗ ┃┃┏┃┓ ┗┛┃┓┛ ┗ ┓┓┛┏┏ ┗━┓┛ ┗┗━━┓━━ 
 ┓┓┗┃┓━┃┗━┗┓┛ ┗┓┏┛ ┃┃┏┃┏┃━┏━┛┛ ┃  ┃┗┗━┗ ┛┛┃┗━┓┛┗┃┏┗━┃━┛┓┛━┏┏┃┗┃┃┏┏┛┏┃┓┓┃┗┗┗┓ ┏┓┛┗┏┓ ┗━┃┛┛┏┓  ┏ ┏┃┏┃━┛ ┗━┗┗┃━┃┓┗━┃━┏┃┛┓━┗━┏┃━┓┓┓┃┗┃┏┓━┗━┓┓┏┛┓
━┓┓┓┃┃┗┛ ┃┛┃ ━━┏┓ ┓━┗┗┃━━━┛┛┓ ┏━┛━┃┛┗┏┃━┗━━┃┓┃ ━┛┗┓┏┓┗┗━━━━┏┏┓ ┏┃┛┗┃┗┛━┛┓ ┏┗┓━ ┓┏┃┃┓ ┃ ┗┓┏┓┓┏┓┏┓┓┏┓┓┗┏┓┏┓┗┗ ━┛┏┃ ┏┃┃┓┓┓ ┃┏  ┃┃┏┏━┛┏┛┓  ┗━┃┛┛
┏┃┏┓┓┏━━┓┛┗┃┓┛┏┏━━┗  ┏┛┛┓┏┛┏┏┏━┓┓┃┃┏┓┏┗━┃┗┏━━┃┗┛ ┃━┛━┓┛┃┗┛┏┗┛┗┓┃┗┓━━━┛┛ ━━┏┛┗ ┛┓┏┛┃ ┗━━┏━━━┓┃┗┛┗━┛┗┓━┛┃┛┗ ┃┏ ┏┃┏┗┛┓┃┏┓┏┗┓┗┃┓┏┗┛━┛┓  ┏┏ ┃┛ ┗┓
┏┗┛┛┏┗┃┗┓━┏┗━━┏┗┛┓┗┓━┗┃┛┃━┓┏━┃┏┗━┃┛┛┏┛┛┓┏┓━┛ ┏━━┏━┛┗━┃━┗┓┏━┓┏┛┃┗┗━━┛┗┓┏┃┛┓┃┏┃ ┗┏┛┏┛   ┃┗┓┏┓┃┗━┓┏━━━┛┛┏┃┓ ┏┗┛┓┛┃┃ ┗━┛┛┗┗┓┛┗┓━━┛┗┗┗┏┓ ┛┃┗━┓┏ ┓
┏┃┗━┓┏┓━┃━┛┗┗┛┃┏┃┗ ┗┓┗┏━┗┗┓┗━┗┓━┏┗┓┓┗━┗━┛┃ ┏┗ ┓ ┗┛━┏━┛┛┏┛┛┓┃┛┏┃━┛┃ ━┛┏┏━━━┗┃┓ ┏┗┓┃┓┓ ┏┏┃┗┛┃┗┓ ┃┗━┓ ┗┃┏┏┓━┓┗┃━━┛┓┓ ┗┛┛┛┏┃┛ ┗┓┏┏┓━┃ ━┓┃━━┗┗┏━┛
┏┛┏┗┛━┛━┗┓━┗━

In [76]:
max(Pipes(data).traverse())

6820

In [77]:
submit_answer(DAY, 1, Out[76])

6820
("That's the right answer!  You are one gold star closer to restoring snow "
 'operations. [Continue to Part Two]')


True

In [58]:
def extrapolate_rev(line):
    nums = list(map(int, line.strip().split(" ")))
    firsts = [nums[0]]

    while any(nums):
        nums = [b-a for a,b in pairwise(nums)]
        firsts.append(nums[0])
    news = [0]
    for b in firsts[::-1][1:]:
        news.append(b-news[-1])
    return news[-1]
    

def extrapolate_all_rev(lines_in):
    return sum(extrapolate_rev(line) for line in lines_in)

In [59]:
extrapolate_all_rev(test_data)

2

In [60]:
extrapolate_all_rev(data)

966

In [61]:
submit_answer(DAY, 2, Out[60])

966
("That's the right answer!  You are one gold star closer to restoring snow "
 'operations.You have completed Day 9! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True